# Importando

In [42]:
import io

import pandas as pd
import requests


import json
pd.set_option('display.max_rows', 100)

# Lendo os dados

Tutorial para criaçao do credentials.json: <br> https://developers.google.com/workspace/guides/create-credentials#desktop

In [8]:
# Código retirado de https://developers.google.com/drive/api/v3/quickstart/python

from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive.file', 'https://www.googleapis.com/auth/drive.metadata', 'https://www.googleapis.com/auth/drive.readonly']
service = None
files = {}

"""Shows basic usage of the Drive v3 API.
Prints the names and ids of the first 10 files the user has access to.
"""
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('drive', 'v3', credentials=creds)

# Call the Drive v3 API
results = service.files().list(
    pageSize=10, fields="nextPageToken, files(id, name)", q="name contains 'Cópia de acervo'").execute()
items = results.get('files', [])

if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        name = item['name']
        if name not in files:
            files[name] = item['id']

files

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=215612286146-7djed3ijn9in6k0kp45484q7uvjkue1q.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A58870%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.metadata+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=VKhXERLHqV8fza3US0xlzp22e8eAUJ&access_type=offline
Files:


{'Cópia de acervo_1980_1989.json': '1bzqZU4EVANPiQhN7q5fk5_dHUmbAtTBq',
 'Cópia de acervo_1556_1899.json': '13kY1eN9suMJv2g3cKo4N9wSxpKD9CBgb',
 'Cópia de acervo_1900_1979.json': '1aPI72OgDF55euwAs1Nb_mxghUK7NNg1c'}

In [14]:
# Download dos arquivos
for file_ in files:
    with open(file_, 'wb') as f:
        file_id = files[file_]
        request = service.files().get_media(fileId=file_id)
        f.write(request.execute())


# Investigando os dados

In [28]:
files_names = list(files.keys())
full_data = []
for file_name in files_names:
    with open(file_name, 'r') as f:
        data = json.loads(f.read())
        full_data.append(data)
    

Vamos investigar o json

In [29]:
print(full_data[0].keys())

dict_keys(['_default'])


In [50]:
print(list(full_data[0]['_default'].keys())[:100])

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100']


Agora sabemos que apesar de estar disposto como um dict, ele é uma lista de documentos.

Agora vamos ver alguns exemplos

In [33]:
full_data[0]['_default']['50000']

{'tipoDocumento': 'Lei',
 'facet-tipoDocumento': 'Legislação::Lei',
 'data': '1983-05-27',
 'urn': 'urn:lex:br;minas.gerais:estadual:lei:1983-05-27;8397',
 'url': 'https://www.lexml.gov.br/urn/urn:lex:br;minas.gerais:estadual:lei:1983-05-27;8397',
 'localidade': 'Minas Gerais',
 'facet-localidade': 'Estados::Minas Gerais',
 'autoridade': 'Estadual',
 'facet-autoridade': 'Estadual',
 'title': 'Lei no 8397, de 27 de maio de 1983',
 'description': 'AUTORIZA O PODER EXECUTIVO A ABRIR CRÉDITO ESPECIAL DE CR$35.000.000,00 À SECRETARIA DE ESTADO DE INDÚSTRIA, COMÉRCIO E TURISMO.',
 'type': 'html',
 'identifier': '005396781'}

In [34]:
full_data[0]['_default']['1']

{'tipoDocumento': ['PL Projeto de Lei',
  'PLC Projeto de Lei',
  'PLC Projeto de Lei',
  'PL Projeto de Lei'],
 'facet-tipoDocumento': ['Proposições Legislativas::Projeto de Lei',
  'Proposições Legislativas::Projeto de Lei',
  'Proposições Legislativas::Projeto de Lei',
  'Proposições Legislativas::Projeto de Lei'],
 'data': ['1980', '1980'],
 'urn': ['urn:lex:br:camara.deputados:projeto.lei;pl:1980;3181',
  'urn:lex:br:senado.federal:projeto.lei;plc:1980;98'],
 'url': "https://www.lexml.gov.br/urn/['urn:lex:br:camara.deputados:projeto.lei;pl:1980;3181', 'urn:lex:br:senado.federal:projeto.lei;plc:1980;98']",
 'localidade': 'Brasil',
 'facet-localidade': 'Brasil',
 'autoridade': ['Câmara dos Deputados', 'Senado Federal'],
 'facet-autoridade': ['Federal::Legislativo::Câmara dos Deputados',
  'Federal::Legislativo::Senado Federal'],
 'title': '[ Projeto de Lei (CD) no 3181/1980 > Projeto de Lei da Câmara no 98/1980 ]',
 'description': 'DISPÕE SOBRE ISENÇÃO DA MULTA PREVIST

Notamos uma falta de de padrão na tipagem dos dados, mas para nossa rápida transformação podemos transformar as lista de forma que peguemos apenas o último valor.

Mas é importante notar que uma análise profunda da semântica desses dados seria necessária para criar uma base útil. <br>
Por exemplo: na autoridade percebemos dois valores distintos, o que pode indicar uma certa importância de manter ambos.

Outra coisa a se fazer é verificar se algum dado possui algum outro campo se não os já vistos:

In [35]:
# Criamos um set fields para armazenar os campos únicos
fields = set()

# Agora percorremos todos os dados para adicionar possíveis novos campos
for data in full_data:
    for i in data['_default']:

        if len(fields) == 0:
            # Carregamos todos os campos do primeiro dado
            fields = data['_default'][i].keys()
        else:
            # Percorremos os campos para adicionar possíveis novos
            for field in data['_default'][i]:
                if field not in fields:
                    fields.add(field)
            

In [36]:
print(fields)

dict_keys(['tipoDocumento', 'facet-tipoDocumento', 'data', 'urn', 'url', 'localidade', 'facet-localidade', 'autoridade', 'facet-autoridade', 'title', 'description', 'type', 'identifier'])


Aparentemente não há nenhum campo além dos já vistos.

# Transformando os dados

Agora, vamos criar um dataframe utilizando os mesmos campos já vistos, mas antes vamos aplicar a lógica supracitada de transformar as listas em registros únicos.

In [37]:
for i, data in enumerate(full_data):
    for document in data['_default'].values():
        for field in document:
            # Verificando se o campo não é uma string e nem nulo
            if type(document[field]) != str and document[field]:
                document[field] = document[field][-1]
    full_data[i] = data

Vamos juntar todos os json em um só

In [40]:
final_data = []
for data in full_data:
    final_data += [data['_default'][i] for i in data['_default']]

In [41]:
# O pandas consegue por si só transformar uma lista de 
df = pd.DataFrame(final_data)

In [43]:
df.head(100)

,tipoDocumento,facet-tipoDocumento,data,urn,url,localidade,facet-localidade,autoridade,facet-autoridade,title,description,type,identifier
0,PL Projeto de Lei,Proposições Legislativas::Projeto de Lei,1980,urn:lex:br:senado.federal:projeto.lei;plc:1980;98,https://www.lexml.gov.br/urn/['urn:lex:br:cama...,Brasil,Brasil,Senado Federal,Federal::Legislativo::Senado Federal,[ Projeto de Lei (CD) no 3181/1980 > Projeto d...,DISPÕE SOBRE ISENÇÃO DA MULTA PREVISTA NO A...,html,010071252
1,PL Projeto de Lei,Proposições Legislativas::Projeto de Lei,1980,urn:lex:br:senado.federal:projeto.lei;plc:1980;94,https://www.lexml.gov.br/urn/['urn:lex:br:cama...,Brasil,Brasil,Senado Federal,Federal::Legislativo::Senado Federal,[ Projeto de Lei (CD) no 3360/1980 > Projeto d...,TRANSFORMA A FUNDAÇÃO FACULDADE CATOLICA DE ...,html,010071387
2,PL Projeto de Lei,Proposições Legislativas::Projeto de Lei,1980,urn:lex:br:senado.federal:projeto.lei;plc:1980;92,https://www.lexml.gov.br/urn/['urn:lex:br:cama...,Brasil,Brasil,Senado Federal,Federal::Legislativo::Senado Federal,[ Projeto de Lei (CD) no 3361/1980 > Projeto d...,DISPÕE SOBRE A INSPEÇÃO E FISCALIZAÇÃO DA...,html,010071454
3,PL Projeto de Lei,Proposições Legislativas::Projeto de Lei,1980,urn:lex:br:senado.federal:projeto.lei;plc:1980;91,https://www.lexml.gov.br/urn/['urn:lex:br:cama...,Brasil,Brasil,Senado Federal,Federal::Legislativo::Senado Federal,[ Projeto de Lei (CD) no 3362/1980 > Projeto d...,AUTORIZA A SUPERINTENDENCIA DO DESENVOLVIMENTO...,html,010071486
4,PL Projeto de Lei,Proposições Legislativas::Projeto de Lei,1980,urn:lex:br:senado.federal:projeto.lei;plc:1980;90,https://www.lexml.gov.br/urn/['urn:lex:br:cama...,Brasil,Brasil,Senado Federal,Federal::Legislativo::Senado Federal,[ Projeto de Lei (CD) no 3510/1980 > Projeto d...,"AUTORIZA A REVERSÃO, A CAIXA ECONOMICA DO EST...",html,010071519
5,PL Projeto de Lei,Proposições Legislativas::Projeto de Lei,1980,urn:lex:br:senado.federal:projeto.lei;plc:1980;88,https://www.lexml.gov.br/urn/['urn:lex:br:cama...,Brasil,Brasil,Senado Federal,Federal::Legislativo::Senado Federal,[ Projeto de Lei (CD) no 3668/1980 > Projeto d...,DISPÕE SOBRE PROVENTOS DE FUNCIONARIOS APOSEN...,html,010071592
6,PL Projeto de Lei,Proposições Legislativas::Projeto de Lei,1980,urn:lex:br:senado.federal:projeto.lei;plc:1980;87,https://www.lexml.gov.br/urn/['urn:lex:br:cama...,Brasil,Brasil,Senado Federal,Federal::Legislativo::Senado Federal,[ Projeto de Lei (CD) no 3898/1980 > Projeto d...,DISPÕE SOBRE O PROVIMENTO DE CARGOS DE JUIZ D...,html,010071630
7,PLP Projeto de Lei Complementar,Proposições Legislativas::Projeto de Lei Com...,1980,urn:lex:br:senado.federal:projeto.lei;plc:1980;86,https://www.lexml.gov.br/urn/['urn:lex:br:cama...,Brasil,Brasil,Senado Federal,Federal::Legislativo::Senado Federal,[ Projeto Lei Complementar (CD) no 153/1980 > ...,ALTERA A REDAÇÃO DO ARTIGO 6 DA LEI COMPLEME...,html,010071668
8,PL Projeto de Lei,Proposições Legislativas::Projeto de Lei,1980,urn:lex:br:senado.federal:projeto.lei;plc:1980;85,https://www.lexml.gov.br/urn/['urn:lex:br:cama...,Brasil,Brasil,Senado Federal,Federal::Legislativo::Senado Federal,[ Projeto de Lei (CD) no 3714/1980 > Projeto d...,CRIA A DECIMA SEGUNDA REGIÃO DA JUSTIÇA DO T...,html,010071704
9,PL Projeto de Lei,Proposições Legislativas::Projeto de Lei,1980,urn:lex:br:senado.federal:projeto.lei;plc:1980;84,https://www.lexml.gov.br/urn/['urn:lex:br:cama...,Brasil,Brasil,Senado Federal,Federal::Legislativo::Senado Federal,[ Projeto de Lei (CD) no 4023/1980 > Projeto d...,AUTORIZA O PODER EXECUTIVO A ABRIR AO MINISTER...,html,010071737


In [44]:
df.tail(100)

,tipoDocumento,facet-tipoDocumento,data,urn,url,localidade,facet-localidade,autoridade,facet-autoridade,title,description,type,identifier
880726,Lei,Legislação::Lei,1979-10-24,urn:lex:br;rio.grande.sul;novo.hamburgo:munici...,https://www.lexml.gov.br/urn/urn:lex:br;rio.gr...,Novo Hamburgo - RS,Municípios::Novo Hamburgo - RS,Municipal,Municipal,"Lei Municipal n° 47, de 24 de Outubro de 1979","DENOMINA ""PRAÇA DR. ROBERTO RECHENMACHER"".",html,004918271
880727,Lei,Legislação::Lei,1979-10-24,urn:lex:br;rio.grande.sul;novo.hamburgo:munici...,https://www.lexml.gov.br/urn/urn:lex:br;rio.gr...,Novo Hamburgo - RS,Municípios::Novo Hamburgo - RS,Municipal,Municipal,"Lei Municipal n° 48, de 24 de Outubro de 1979",DISPÕE SOBRE AUTORIZAÇÃO PARA ABERTURA DE C...,html,004918272
880728,Lei,Legislação::Lei,1979-11-08,urn:lex:br;rio.grande.sul;novo.hamburgo:munici...,https://www.lexml.gov.br/urn/urn:lex:br;rio.gr...,Novo Hamburgo - RS,Municípios::Novo Hamburgo - RS,Municipal,Municipal,"Lei Municipal n° 49, de 08 de Novembro de 1979",DISPÕE SOBRE AUTORIZAÇÃO PARA ABERTURA DE C...,html,004918273
880729,Lei,Legislação::Lei,1979-03-21,urn:lex:br;rio.grande.sul;novo.hamburgo:munici...,https://www.lexml.gov.br/urn/urn:lex:br;rio.gr...,Novo Hamburgo - RS,Municípios::Novo Hamburgo - RS,Municipal,Municipal,"Lei Municipal n° 5, de 21 de Março de 1979",RECONHECE DE UTILIDADE PÚBLICA A SOCIEDADE DE...,html,004918274
880730,Lei,Legislação::Lei,1979-11-08,urn:lex:br;rio.grande.sul;novo.hamburgo:munici...,https://www.lexml.gov.br/urn/urn:lex:br;rio.gr...,Novo Hamburgo - RS,Municípios::Novo Hamburgo - RS,Municipal,Municipal,"Lei Municipal n° 50, de 08 de Novembro de 1979",DISPÕE SOBRE AUTORIZAÇÃO PARA ABERTURA DE C...,html,004918275
880731,Lei,Legislação::Lei,1979-11-20,urn:lex:br;rio.grande.sul;novo.hamburgo:munici...,https://www.lexml.gov.br/urn/urn:lex:br;rio.gr...,Novo Hamburgo - RS,Municípios::Novo Hamburgo - RS,Municipal,Municipal,"Lei Municipal n° 51, de 20 de Novembro de 1979",CONCEDE ABONO DE NATAL E DÁ OUTRAS PROVIDÊNC...,html,004918276
880732,Lei,Legislação::Lei,1979-11-20,urn:lex:br;rio.grande.sul;novo.hamburgo:munici...,https://www.lexml.gov.br/urn/urn:lex:br;rio.gr...,Novo Hamburgo - RS,Municípios::Novo Hamburgo - RS,Municipal,Municipal,"Lei Municipal n° 52, de 20 de Novembro de 1979",CONCEDE ABONO DE NATAL E DÁ OUTRAS PROVIDÊNC...,html,004918277
880733,Lei,Legislação::Lei,1979-11-23,urn:lex:br;rio.grande.sul;novo.hamburgo:munici...,https://www.lexml.gov.br/urn/urn:lex:br;rio.gr...,Novo Hamburgo - RS,Municípios::Novo Hamburgo - RS,Municipal,Municipal,"Lei Municipal n° 53, de 23 de Novembro de 1979","DENOMINA ""PRAÇA EWALDO DÖERNER"" UMA PRAÇA D...",html,004918278
880734,Lei,Legislação::Lei,1979-11-26,urn:lex:br;rio.grande.sul;novo.hamburgo:munici...,https://www.lexml.gov.br/urn/urn:lex:br;rio.gr...,Novo Hamburgo - RS,Municípios::Novo Hamburgo - RS,Municipal,Municipal,"Lei Municipal n° 54, de 26 de Novembro de 1979",DÁ DENOMINAÇÃO AOS NÚCLEOS COMUNITÁRIOS.,html,004918279
880735,Lei,Legislação::Lei,1979-11-26,urn:lex:br;rio.grande.sul;novo.hamburgo:munici...,https://www.lexml.gov.br/urn/urn:lex:br;rio.gr...,Novo Hamburgo - RS,Municípios::Novo Hamburgo - RS,Municipal,Municipal,"Lei Municipal n° 55, de 26 de Novembro de 1979",AUTORIZA O PREFEITO MUNICIPAL A FIRMAR CONVÊN...,html,004918280


Agora, quase temos uma base, porém uma olhada rápida me deu 3 ideias para melhorar a qualidade da base:<br>
- O campo data pode ter um padrão: ser sempre YYYY-mm-dd
- O title possui o chaves em alguns registros, vamos retirar para ficar mais legível
- Podemos renomear os campos para fazerem mais sentido:
    - tipoDocumento: TipoDocumento
    - facet-tipoDocumento: FacetTipoDocumento
    - data: DataDocumento
    - urn: Urn
    - url: Url
    - localidade: Localidade
    - facet-localidade: FacetLocalidade
    - autoridade: Autoridade
    - facet-autoridade: FacetAutoridade
    - title: Titulo
    - description: Descricao
    - type: Tipo
    - identifier: Identificador
<br><br>
Outra idéia seria dividir os facets em hierarquias, mas isso requer conhecimento específico para poder saber ao certo a hierarquia correta.

In [45]:
# Arrumando a data
new_data = []
for i, data in enumerate(df['data']):
    if len(data) == 4:
        data = data+'-01-01'
    new_data.append(data)
    
df['data'] = new_data

In [46]:
# Arrumando o title
new_title = []
for i, title in enumerate(df['title']):
    if title and title[0] == '[':
        title = title.replace('[ ', '').replace(' ]', '')
    new_title.append(title)

df['title'] = new_title

In [47]:
df.columns = ['TipoDocumento', 'FacetTipoDocumento', 'DataDocumento', 'Urn', 'Url', 'Localidade', 'FacetLocalidade', 'Autoridade', 'FacetAutoridade', 'Titulo', 'Descricao', 'Tipo', 'Identificador']

In [48]:
df

,TipoDocumento,FacetTipoDocumento,DataDocumento,Urn,Url,Localidade,FacetLocalidade,Autoridade,FacetAutoridade,Titulo,Descricao,Tipo,Identificador
0,PL Projeto de Lei,Proposições Legislativas::Projeto de Lei,1980-01-01,urn:lex:br:senado.federal:projeto.lei;plc:1980;98,https://www.lexml.gov.br/urn/['urn:lex:br:cama...,Brasil,Brasil,Senado Federal,Federal::Legislativo::Senado Federal,Projeto de Lei (CD) no 3181/1980 > Projeto de ...,DISPÕE SOBRE ISENÇÃO DA MULTA PREVISTA NO A...,html,010071252
1,PL Projeto de Lei,Proposições Legislativas::Projeto de Lei,1980-01-01,urn:lex:br:senado.federal:projeto.lei;plc:1980;94,https://www.lexml.gov.br/urn/['urn:lex:br:cama...,Brasil,Brasil,Senado Federal,Federal::Legislativo::Senado Federal,Projeto de Lei (CD) no 3360/1980 > Projeto de ...,TRANSFORMA A FUNDAÇÃO FACULDADE CATOLICA DE ...,html,010071387
2,PL Projeto de Lei,Proposições Legislativas::Projeto de Lei,1980-01-01,urn:lex:br:senado.federal:projeto.lei;plc:1980;92,https://www.lexml.gov.br/urn/['urn:lex:br:cama...,Brasil,Brasil,Senado Federal,Federal::Legislativo::Senado Federal,Projeto de Lei (CD) no 3361/1980 > Projeto de ...,DISPÕE SOBRE A INSPEÇÃO E FISCALIZAÇÃO DA...,html,010071454
3,PL Projeto de Lei,Proposições Legislativas::Projeto de Lei,1980-01-01,urn:lex:br:senado.federal:projeto.lei;plc:1980;91,https://www.lexml.gov.br/urn/['urn:lex:br:cama...,Brasil,Brasil,Senado Federal,Federal::Legislativo::Senado Federal,Projeto de Lei (CD) no 3362/1980 > Projeto de ...,AUTORIZA A SUPERINTENDENCIA DO DESENVOLVIMENTO...,html,010071486
4,PL Projeto de Lei,Proposições Legislativas::Projeto de Lei,1980-01-01,urn:lex:br:senado.federal:projeto.lei;plc:1980;90,https://www.lexml.gov.br/urn/['urn:lex:br:cama...,Brasil,Brasil,Senado Federal,Federal::Legislativo::Senado Federal,Projeto de Lei (CD) no 3510/1980 > Projeto de ...,"AUTORIZA A REVERSÃO, A CAIXA ECONOMICA DO EST...",html,010071519
...,...,...,...,...,...,...,...,...,...,...,...,...,...
880821,MS Acórdão,Jurisprudência::Acórdão,1979-02-20,urn:lex:br:tribunal.superior.eleitoral;plenari...,https://www.lexml.gov.br/urn/urn:lex:br:tribun...,Brasil,Brasil,Tribunal Superior Eleitoral. Plenário,Federal::Judiciário::Tribunal Superior Eleito...,Acórdão MS de 20/02/1979 (Processo MS no 495),CONCORRER AO PREENCHIMENTO DE VAGA NO SERVICO ...,html,004960319
880822,AG Acórdão,Jurisprudência::Acórdão,1979-08-16,urn:lex:br:tribunal.superior.eleitoral;plenari...,https://www.lexml.gov.br/urn/urn:lex:br:tribun...,Brasil,Brasil,Tribunal Superior Eleitoral. Plenário,Federal::Judiciário::Tribunal Superior Eleito...,Acórdão AG de 16/08/1979 (Processo AG no 5165),"DA-SE PROVIMENTO A AGRAVO DE INSTRUMENTO, PARA...",html,004960662
880823,REspe Acórdão,Jurisprudência::Acórdão,1979-04-03,urn:lex:br:tribunal.superior.eleitoral;plenari...,https://www.lexml.gov.br/urn/urn:lex:br:tribun...,Brasil,Brasil,Tribunal Superior Eleitoral. Plenário,Federal::Judiciário::Tribunal Superior Eleito...,Acórdão RESPE de 03/04/1979 (Processo RESPE ...,RECURSO ESPECIAL. CONSULTA AO TRE. NAO CABE RE...,html,004961411
880824,REspe Acórdão,Jurisprudência::Acórdão,1979-05-24,urn:lex:br:tribunal.superior.eleitoral;plenari...,https://www.lexml.gov.br/urn/urn:lex:br:tribun...,Brasil,Brasil,Tribunal Superior Eleitoral. Plenário,Federal::Judiciário::Tribunal Superior Eleito...,Acórdão RESPE de 24/05/1979 (Processo RESPE ...,- JUIZ ELEITORAL. - SEU AFASTAMENTO DEFINITIVO...,html,004961424


# Exportando

Simplesmente usamos o método to_csv do pandas para exportar a base.

In [49]:
df.head(300).to_csv('data.csv', index=False)